# Les modèles pour l’apprentissage supervisé

Pour mémoire, les modèles pour l’apprentissage supervisé sont classés en fonction de la tâche à réaliser :
- La **régression**, quand la prévision porte sur une valeur continue comme le salaire d’un·e étudiant·e en fin de formation ;
- ou la **classification** lorsqu’il s’agit de prédire l’appartenance à une classe.

Certains modèles sont réservés à l’une ou l’autre de ces tâches quand les autres peuvent servir aux deux. Dans ce calepin, nous parcourons les plus populaires sans pour autant rentrer dans les détails.

Avant de commencer, chargeons les librairies nécessaires et constituons des ensembles de données pour les différentes tâches à traiter :

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.compose import make_column_selector as col_selector
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

###################
# Regression task #
###################

# for deterministic purposes
np.random.seed(0)

# hundred points
X = np.random.rand(100, 1)

# polynomial function: y = 2 - 3x + 3x^2 + 5x^3
y = 2 - 3 * X + 3 * X ** 2 + 5 * X ** 3 + np.random.rand(100, 1)

#######################
# Classification task #
#######################

# penguin census
df = pd.read_csv("./data/penguin-census.csv").dropna()

# variables
target = "species"
features = ["flipper_length_mm", "body_mass_g", "sex"]

# dataset
X_b = df[features]
y_b = df[target]

# selectors
num_col_selector = col_selector(dtype_exclude=object)
cat_col_selector = col_selector(dtype_include=object)

# num & cat cols
num_cols = num_col_selector(X_b)
cat_cols = cat_col_selector(X_b)

# preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ("num-transformers", StandardScaler(), num_cols),
        ("cat-transformers", OneHotEncoder(handle_unknown="ignore"), cat_cols),
    ]
)

## Avant-propos

Les modèles d’apprentissage de *Scikit-Learn*, programmés en différentes classes, sont conçus autour du même esprit de cohésion. Chaque classe expose notamment les mêmes méthodes suivantes :
- `fit()` pour ajuster le modèle sur des données distribuées dans deux paramètres `x` et `y` ;
- `predict()` pour effectuer des prédictions sur des données ;
- `score()` pour obtenir une évaluation des prédictions obtenues.

## Les modèles pour les tâches de régression

### La régression linéaire

Sans doute la méthode la plus populaire en apprentissage supervisé, la régression linéaire consiste à trouver la fonction affine $y$ d’une variable explicative $x$. Elle permet de comprendre rapidement la relation entre deux variables. Pour autant, sa sensibilité aux données aberrantes et aux *outliers* peut l’empêcher d’observer une corrélation ; elle se révèle alors une hypothèse trop simple pour les données. On parle alors de sous-ajustement (*underfitting*).

In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
_ = model.fit(X, y)

Une fois le modèle entraîné, il est possible d’accéder au coefficient directeur de la droite et à son ordonnée à l’origine grâce aux attributs `coef_` et `intercept_` :

In [ ]:
print(model.coef_, model.intercept_)

Tout comme en connaître la précision, ici grâce au coefficient de détermination $R^2$ :

In [ ]:
model.score(X, y)

L’interprétation des résultats d’une régression linéaire est extrêmement simple et généralement immédiate :

In [ ]:
_ = sns.regplot(x=X, y=y)

### La régression polynomiale

Le modèle de régression linéaire par ajustement affine ne s’est pas révélé une bonne hypothèse pour nos données d’entrée. Pour rappel, nous avions utilisé un polynôme de degré 3 pour générer les coordonnées en $y$.

Lorsque, en visualisant les données, on observe une relation non-linéaire, l’hypothèse est qu’il doit exister d’autres caractéristiques pour l’expliquer que simplement $X$. *Scikit-Learn* met à disposition une classe `PolynomialFeatures` pour les ajouter à $X$ en fonction du polynôme considéré dans un paramètre `degree` :

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

model = make_pipeline(
    PolynomialFeatures(degree=3),
    LinearRegression(),
)
model.fit(X, y)
y_pred = model.predict(X)

Le modèle s’ajuste bien mieux aux données que la précédente régression linéaire, ce qui se traduit immédiatement dans une augmentation de son *$R^2$ score* :

In [ ]:
ax = sns.scatterplot(x=X[:, 0], y=y[:, 0])
ax.plot(sorted(X), sorted(y_pred))
_ = ax.set_title(f"R2 score = {model.score(X, y):.2f}")

Il serait tentant de chercher à améliorer le *$R^2$ score* en augmentant le degré du polynôme, mais le risque serait d’ajuster trop bien le modèle (*overfitting*) aux données d’entraînement qu’il ne parviendrait plus à obtenir de bonnes prédictions sur des données nouvelles :

In [ ]:
model = make_pipeline(
    PolynomialFeatures(degree=15),
    LinearRegression(),
)
model.fit(X, y)

y_pred = model.predict(X)

ax = sns.scatterplot(x=X[:, 0], y=y[:, 0])
ax.plot(sorted(X), sorted(y_pred))
_ = ax.set_title(f"R2 score = {model.score(X, y):.2f}")

La courbe semble plus fortement attirée par les données avec un polynôme de très haut degré, ce qui, en quelque sorte, la rend dépendante vis-à-vis d’elles. Il faut s’attendre à voir le *$R^2$ score* diminuer sensiblement sur des données nouvelles :

In [ ]:
np.random.seed(1)

X_prime = np.random.rand(100, 1)
y_prime = 2 - 3 * X_prime + 3 * X_prime ** 2 + 5 * X_prime ** 3 + np.random.rand(100, 1)

y_pred = model.predict(X_prime)

ax = sns.scatterplot(x=X_prime[:, 0], y=y_prime[:, 0])
ax.plot(sorted(X_prime), sorted(y_pred))
_ = ax.set_title(f"R2 score = {model.score(X_prime, y_prime):.2f}")

## Les modèles pour les tâches de classification

### Les K plus proches voisins

Le modèle de recherche des $k$ plus proches voisins (*K-Nearest Neighbors*) consiste à rechercher, pour toute nouvelle entrée, les $k$ entrées les plus proches dans la base d’apprentissage. Il est par ailleurs possible de préciser $k$ grâce à un paramètre `n_neighbors`.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

model = make_pipeline(preprocessor, KNeighborsClassifier(n_neighbors=3))
_ = model.fit(X_b, y_b)

Parmi les attributs intéressants de la classe, citons `classes_` et `feature_names_in_` qui permettent de garder une trace des différentes modalités ainsi que des variables explicatives utilisées dans le modèle :

In [ ]:
print(
    f"Modalités : {model.classes_}",
    f"Variables explicatives : {model.feature_names_in_}",
    sep="\n"
)

Le score obtenu est le taux de prédictions exactes sur l’ensemble du jeu d’apprentissage :

In [ ]:
model.score(X_b, y_b)

### La régression logistique

L’algorithme de régression logistique est très souvent employé dans les tâches de classification car il peut estimer rapidement la probabilité qu’une observation appartienne à une modalité particulière. Un paramètre `class_weight` permet de transmettre un dictionnaire de poids pour les modalités afin de pondérer la classification, ce qui peut se révéler utile pour corriger un biais de représentativité. Dans le cas contraire, toutes les étiquettes sont réputées avoir un poids de 1. Il est également possible de modifier l’algorithme utilisé par défaut pour l’optimisation de la régression avec un paramètre `solver`. Notons également un paramètre `max_iter` fixé par défaut à 100, qu’il peut être utilse d’augmenter afin de faire disparaître un avertissement `ConvergenceWarning` :

In [ ]:
from sklearn.linear_model import LogisticRegression

weights = {
    "Adelie": .4,
    "Gentoo": .2,
    "Chinstrap": .4
}

model = make_pipeline(
    preprocessor,
    LogisticRegression(
        max_iter=100,
        class_weight=weights,
        solver="liblinear"
    )
)
model.fit(X_b, y_b)
model.score(X_b, y_b)

### Les machines à vecteurs de support

### La classification naïve bayésienne

## Les modèles mixtes

### Les arbres de décision

### Les forêts d’arbres décisionnels

### Les réseaux de neurones